In [ ]:
class POT:
    
    def __init__(self):
        
    
    def cdf(x, thr, mu, kmax, loc, scale):
        """Calcula la función de distribución acumulada (CDF, cumulative density function) de una distribución POT (peaks-over-threshold) compuesta por una Poisson y una exponencial.

        Parámetros:
        -----------
        x:         array or list (n,). Valor/es de la variable para los que calcular su probabilidad de no excedencia
        thr:       float. Umbral que define los eventos considerados como extremos en la POT
        mu:        float. Parámetro de ocurrencia media de la distribución de Poisson
        kmax:      float. Nº máximo de ocurrencias de evento por año
        loc:       float. Parámetro de localización de la distribución exponencial
        scale:     float. Parámetro de escala de la distribución exponencial

        Salida:
        -------
        cdf:       array (n,). Probabilidad de no excedencia de cada uno de los valores de 'x'
        """

        # asegurar que 'x' es un array
        x = np.array(x)
        x -= thr
        # 'array' donde guardar la probabilidad acumulada
        cdf = np.zeros_like(x)
        for k in range(0, kmax + 1): # para cada 'k': nº de eventos anuales
            cdf += stats.poisson.pmf(k, mu) * stats.expon.cdf(x, *parsExp)**k

        return cdf
    
    def pdf(x, thr, mu, kmax, loc, scale):
        """Calcula la función de densidad (PDF, probability distribution function) de una distribución POT (peaks-over-threshold) compuesta por una Poisson y una exponencial.

        Parámetros:
        -----------
        x:         array or list (n,). Valor/es de la variable para los que calcular su probabilidad de no excedencia
        thr:       float. Umbral que define los eventos considerados como extremos en la POT
        mu:        float. Parámetro de ocurrencia media de la distribución de Poisson
        kmax:      float. Nº máximo de ocurrencias de evento por año
        loc:       float. Parámetro de localización de la distribución exponencial
        scale:     float. Parámetro de escala de la distribución exponencial

        Salida:
        -------
        pdf:       array (n,). Probabilidad de cada uno de los valores de 'x'
        """

        # asegurar que 'x' es un array
        x = np.array(x)
        x -= thr
        # 'array' donde guardar la probabilidad acumulada
        pdf = np.zeros_like(x)
        for k in range(0, kmax + 1): # para cada 'k': nº de eventos anuales
            pdf += stats.poisson.pmf(k, mu) * k * stats.expon.cdf(x, *parsExp)**(k - 1) * stats.expon.pdf(x, *parsExp)

        return pdf
    
    def ppf(q, thr, mu, kmax, loc, scale):
        """Calcula el cuantil asociado a una probabilidad de no excedencia de una distribución POT (peaks-over-threshold) compuesta por una Poisson y una exponencial.

        Parámetros:
        -----------
        q:         array or list (n,). Valor/es de probabilidad de no excedencia
        thr:       float. Umbral que define los eventos considerados como extremos en la POT
        mu:        float. Parámetro de ocurrencia media de la distribución de Poisson
        kmax:      float. Nº máximo de ocurrencias de evento por año
        loc:       float. Parámetro de localización de la distribución exponencial
        scale:     float. Parámetro de escala de la distribución exponencial

        Salida:
        -------
        x:       array (n,). Cuantil asociado a cada probabilidad de 'q'
        """

        # asegurar que 'q' es un array
        q = np.array(q)

        # array de resultados
        x = np.zeros_like(q)

        # calcular el cuantil para cada probabilidad de no excedencia
        # ecuación: PMFpoiss(kmax) * CDFexp(y)**kmax + ... + PMFpoiss(2) * CDFexp(y)**2 + PMFpoiss(1) * CDFexp(y) - q = 0
        for i, qx in enumerate(q):
            # coeficientes de la ecuación lineal: [PMFpoiss(kmax), ..., PMFpoiss(2), PMFpoiss(1), -q]
            p = np.array([stats.poisson.pmf(k, mu) for k in range(0, kmax + 1)[::-1]])
            p = np.concatenate((p, np.array([-qx])))
            # raíces de la ecuación: CDFexp(y)
            roots = np.roots(p)
            # encontrar la raíz real con valor positivo
            for root in np.roots(p):
                if (np.imag(root) == 0) & (np.real(root) > 0):
                    qy = np.real(root)
                    break

            # valor de exceso asociado a dicho cuantil
            y = stats.expon.ppf(qy, *parsExp)
            # convertir a la variable original
            x[i] = y + thr

        return x